In [4]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer

stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 
             'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 
             'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 
             'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 
             'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 
             'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 
             'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 
             'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 
             'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 
             'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 
             'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
             ]

df = pd.read_pickle('jiujitsu videos links.pkl')
for i, row in df.iterrows():
    if '천' in row['view_num']:
        row['view_num'] = float(row['view_num'].strip('천'))*1000
    elif '만' in row['view_num']:
        row['view_num'] = float(row['view_num'].strip('만'))*10000
    else:
        row['view_num'] = float(row['view_num'])

In [7]:
# tokenization
text = df[['title']].copy()
text['tokenized_title'] = text.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)
text['tokenized_title'] = text['tokenized_title'].apply(lambda x: [word.lower() for word in x if word.lower() not in (stopwords)])
text['tokenized_title'] = text['tokenized_title'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 단어 인덱싱을 위한 단어 딕셔너리
words_list = {}
i=0
for i, row in text.iterrows():
    for word in row['tokenized_title']:
        if word not in words_list:
            words_list[word] = i 
            i += 1
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text['tokenized_title'][i])
    detokenized_doc.append(t)
text['detokenized_title'] = detokenized_doc
text = text.drop(['title'], axis=1)
new_df = pd.concat([df, text], axis=1)
new_df.to_csv('tokenized title.csv')

In [29]:
title_list = text['detokenized_title'].to_list()
vect = TfidfVectorizer(ngram_range=(1,3))
vect.fit_transform(title_list)
vocab_dict = vect.vocabulary_
vocab_dict = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
vocab_df = pd.DataFrame(vocab_dict)
vocab_df.to_csv('countvectorizer.csv')
vocab_df
# stop_list = []
# for voca in vocab_dict:
#     if voca[0] in stopwords:
#         stop_list.append(voca)

,0,1
0,zwevegem belgium,35185
1,zwevegem,35184
2,zorro pass,35183
3,zorro,35182
4,zombie apocalypse,35181
...,...,...
35181,000 miles arctic,4
35182,000 miles,3
35183,000 giveaway chance,2
35184,000 giveaway,1


In [8]:
new_df[:10]

,title,view_num,url,tokenized_title,detokenized_title
0,Old school Lapel tricks -Kimura,180,https://www.youtube.com/watch?v=QM9Qp1kYbkE,"[old, school, lapel, trick, -kimura]",old school lapel trick -kimura
1,Old school Americana from inside the Guard,439,https://www.youtube.com/watch?v=LQXt0A8gJ00,"[old, school, americana, inside, guard]",old school americana inside guard
2,Old school lasso guard pass,460,https://www.youtube.com/watch?v=V9o13Fl3cYs,"[old, school, lasso, guard, pass]",old school lasso guard pass
3,OLD SCHOOL EZEKIAL CHOKE,547,https://www.youtube.com/watch?v=bRjP5Ti8Y3k,"[old, school, ezekial, choke]",old school ezekial choke
4,Old School Foot lock from side control,500,https://www.youtube.com/watch?v=HINWYC9smg8,"[old, school, foot, lock, side, control]",old school foot lock side control
5,Pass the guard defense,327,https://www.youtube.com/watch?v=v8HpzeFIlMQ,"[pass, guard, defense]",pass guard defense
6,ARMBAR ATTACK,541,https://www.youtube.com/watch?v=DM4_azHzm78,"[armbar, attack]",armbar attack
7,Attack from knee on the belly,434,https://www.youtube.com/watch?v=vlDBqhIhw2A,"[attack, knee, belly]",attack knee belly
8,Old School Spider Guard Converted to Single X ...,453,https://www.youtube.com/watch?v=PDadmduhZeY,"[old, school, spider, guard, convert, single, ...",old school spider guard convert single x guard...
9,Butterfly Guard attack Old School,620,https://www.youtube.com/watch?v=y_ZGd1xW_LA,"[butterfly, guard, attack, old, school]",butterfly guard attack old school


In [27]:
# for i, row in new_df.iterrows():
#     tokens = row['tokenized_title']
#     if 'guard' in tokens:

#  Positions
guard_ = new_df[new_df['tokenized_title'].apply(lambda x: 'guard' in x and 'pass' not in x)].copy()
side_ = new_df[new_df['tokenized_title'].apply(lambda x: 'side' in x)].copy()
top_ = new_df[new_df['tokenized_title'].apply(lambda x: 'top' in x)].copy()
north_south_ = new_df[new_df['tokenized_title'].apply(lambda x: 'north' in x or 'south' in x)].copy()
back_ = new_df[new_df['tokenized_title'].apply(lambda x: 'back' in x)].copy()

# Transitions
pass_ = new_df[new_df['tokenized_title'].apply(lambda x: 'pass' in x)].copy()
sweep_ = new_df[new_df['tokenized_title'].apply(lambda x: 'sweep' in x)].copy()

# Escapes
escape_ = new_df[new_df['tokenized_title'].apply(lambda x: 'escape' in x)].copy()

# Submissions
submission_ = new_df[new_df['tokenized_title'].apply(lambda x: 'submission' in x)].copy()
# Arm-Locks
americana_ = new_df[new_df['tokenized_title'].apply(lambda x: 'americana' in x)].copy()
kimura_ = new_df[new_df['tokenized_title'].apply(lambda x: 'kimura' in x)].copy()
armbar_ = new_df[new_df['tokenized_title'].apply(lambda x: 'arm' in x and 'bar' in x)].copy()
omoplata_ = new_df[new_df['tokenized_title'].apply(lambda x: 'omoplata' in x)].copy()

# Chokes
choke_ = new_df[new_df['tokenized_title'].apply(lambda x: 'choke' in x)].copy()
collar_choke_ = new_df[new_df['tokenized_title'].apply(lambda x: 'collar' in x and 'choke' in x)].copy()
guillotine_choke_ = new_df[new_df['tokenized_title'].apply(lambda x: 'guilloti' in x)].copy()
triangle_choke_ = new_df[new_df['tokenized_title'].apply(lambda x: 'triangle' in x)].copy()

# Leg-Locks
ankle_lock_ = new_df[new_df['tokenized_title'].apply(lambda x: 'ankle' in x and 'lock' in x)].copy()
knee_bar_ = new_df[new_df['tokenized_title'].apply(lambda x: 'knee' in x and 'bar' in x)].copy()
heel_hook_ = new_df[new_df['tokenized_title'].apply(lambda x: 'heel' in x and 'hook' in x)].copy()

# Solo-Drills
drill_ = new_df[new_df['tokenized_title'].apply(lambda x: 'drill' in x)].copy()



In [28]:
sweep_

,title,view_num,url,tokenized_title,detokenized_title
8,Old School Spider Guard Converted to Single X ...,453,https://www.youtube.com/watch?v=PDadmduhZeY,"[old, school, spider, guard, convert, single, ...",old school spider guard convert single x guard...
12,Guard Sweeps Tips,407,https://www.youtube.com/watch?v=cRMrHaMk2iw,"[guard, sweep, tip]",guard sweep tip
26,Sweep from spider and lasso guard,438,https://www.youtube.com/watch?v=fgEfDJTglL0,"[sweep, spider, lasso, guard]",sweep spider lasso guard
27,Lasso guard sweep secret & triangle,672,https://www.youtube.com/watch?v=dbM8trCqfCU,"[lasso, guard, sweep, secret, &, triangle]",lasso guard sweep secret & triangle
29,Spider Guard & X guard sweeps,477,https://www.youtube.com/watch?v=nDPf8EYwTOI,"[spider, guard, &, x, guard, sweep]",spider guard & x guard sweep
...,...,...,...,...,...
5261,BJJ Half Butterfly Sweep,3000,https://www.youtube.com/watch?v=RwVmtLFJVn0,"[bjj, half, butterfly, sweep]",bjj half butterfly sweep
5263,BJJ Butterfly Shin In Sweep Calf Slicer,1700,https://www.youtube.com/watch?v=8-lIU99CSgs,"[bjj, butterfly, shin, sweep, calf, slicer]",bjj butterfly shin sweep calf slicer
5264,BJJ Basic X Guard Entry and Ankle Sweep,4200,https://www.youtube.com/watch?v=wdkRWJkGX9o,"[bjj, basic, x, guard, entry, ankle, sweep]",bjj basic x guard entry ankle sweep
5265,BJJ Cross Grip Sweep & Back Take,2000,https://www.youtube.com/watch?v=1d-9ptKzOTE,"[bjj, cross, grip, sweep, &, back, take]",bjj cross grip sweep & back take
